# Learning Box Embeddings with Example

This tutorial outlines the different functionalities available within the Box Embeddings package

### 0. Installing the package on your machine

*If you have the repo cloned*
```
pip install --editable . --user
```

### A. Initialize a box tensor and check its parameters

#### Standard Box Tensor
To represent a Tensor as a Box, we use the class `BoxTensor`. The necessary parameter is `data` (a tensor)

In [72]:
from box_embeddings.parameterizations.box_tensor import *

# Let's create a toy example
x_min = [-2.0]*50
x_max = [0.0]*50
data_x = torch.tensor([x_min, x_max])
box_1 = BoxTensor(data_x)
box_1

BoxTensor(tensor([[-2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2., -2.,
         -2., -2., -2., -2., -2., -2., -2., -2.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]))

We can use several methods to look at the parameters of our box, such as

In [73]:
# Lower left coordinate
print(box.z)
# Top right coordinate
print(box.Z)
# Center coordinate
print(box.centre)


tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250, 0.1111,
        0.1000, 0.0909, 0.0833, 0.0769, 0.0714, 0.0667, 0.0625, 0.0588, 0.0556,
        0.0526, 0.0500, 0.0476, 0.0455, 0.0435, 0.0417, 0.0400, 0.0385, 0.0370,
        0.0357, 0.0345, 0.0333, 0.0323, 0.0312, 0.0303, 0.0294, 0.0286, 0.0278,
        0.0270, 0.0263, 0.0256, 0.0250, 0.0244, 0.0238, 0.0233, 0.0227, 0.0222,
        0.0217, 0.0213, 0.0208, 0.0204, 0.0200, 0.0196, 0.0192, 0.0189, 0.0185,
        0.0182, 0.0179, 0.0175, 0.0172, 0.0169, 0.0167, 0.0164, 0.0161, 0.0159,
        0.0156, 0.0154, 0.0152, 0.0149, 0.0147, 0.0145, 0.0143, 0.0141, 0.0139,
        0.0137, 0.0135, 0.0133, 0.0132, 0.0130, 0.0128, 0.0127, 0.0125, 0.0123,
        0.0122, 0.0120, 0.0119, 0.0118, 0.0116, 0.0115, 0.0114, 0.0112, 0.0111,
        0.0110, 0.0109, 0.0108, 0.0106, 0.0105, 0.0104, 0.0103, 0.0102, 0.0101,
        0.0100])
tensor([2.9900, 2.9899, 2.9898, 2.9897, 2.9896, 2.9895, 2.9894, 2.9892, 2.9891,
        2.9890, 2.9889,

Let's broadcast our box to a new shape.. Broadcasting is often needed for different arithmetic operations. The function we
will use is `broadcast()`, and the required parameter is `target_shape=()`, which specify the new shape
for the box. This is very similar to `numpy.broadcast_to()`

In [74]:
data = torch.tensor([[[1, 2, 3], [3, 4, 6]],
          [[5, 6, 8], [7, 9, 5]]])
box = BoxTensor(data)
print('previous shape is:', box.box_shape)
box.broadcast(target_shape=(1, 2, 3))
print('after broadcasting:', box.box_shape)

previous shape is: (2, 3)
after broadcasting: (1, 2, 3)


#### Delta Box Tensor
We can initialize a box with additional requirement, such as forcing `Z=z + delta` using the method `MinDeltaBoxTensor`

In [42]:
from box_embeddings.parameterizations.delta_box_tensor import *
data = torch.tensor([[1, 2, 3], [3, 4, 6]])
delta_box = MinDeltaBoxTensor(data)
delta_box.z
delta_box.beta

1.0

#### Other Parameterization methods


In [68]:
# TODO
from box_embeddings.parameterizations.sigmoid_box_tensor import *
from box_embeddings.parameterizations.tanh_box_tensor import *

#### Uniform Box
Using the function `uniform_boxes`, we can create several uniform boxes specified by `num_boxes` such that each of
them is inside the bounding box defined by `(minimum,maximum)` in each dimension. The defaults are `minimum=0.0`
and `maximum=1.0`. We can also specify `(delta_min, delta_max)`. The defaults are `delta_min=0.01` and `delta_max=0.5`

In [23]:
# Initialize
from box_embeddings.initializations.uniform_boxes import *
uniform = uniform_boxes(dimensions = 2, num_boxes = 3, minimum = 1.0, maximum = 4.0)

This function returns a tuple of `(z, Z)`, where `z` and `Z` are tensors. `z` are the lower left coordinates of
the 3 boxes in this example. Similarly, `Z` are the top right coordinates.

In [25]:
# Return (z,Z)
uniform

(tensor([[2.1434, 3.3852],
         [1.9863, 2.2522],
         [1.3006, 3.7267]], dtype=torch.float64),
 tensor([[2.4814, 3.4371],
         [2.0022, 2.2938],
         [1.5343, 3.7707]], dtype=torch.float64))

### 2. Box Volume
To calculate the volume of a box, we can use either the `soft_volume`, or the Bessel volume via `bessel_volume_approx`.
To ensure numerical stability, we can use the log version via `log_soft_volume` or `log_bessel_volume_approx`

In [77]:
from box_embeddings.modules.volume.soft_volume import soft_volume, log_soft_volume
from box_embeddings.modules.volume.bessel_volume import bessel_volume_approx, log_bessel_volume_approx

# Create data as tensors, and initialize a box
x_min = [-2.0]*100
x_max = [0.0]*100
data_x = torch.tensor([x_min, x_max])
box_1 = BoxTensor(data_x)

# Soft volume
print(soft_volume(box_1))

# Log Soft volume
print(log_soft_volume(box_1))

tensor(2.4414e+16)
tensor(37.7339)


### 3. Box Intersection

To calculate the intersection of two boxes (which yields a box), we can use either `hard_intersection` or
`gumbel_intersection`

In [78]:
from box_embeddings.modules.intersection import hard_intersection, gumbel_intersection

# Create data as tensors, and initialize two boxes, box_1 and box_2
x_min = [-2.0]*100
x_max = [0.0]*100
data_x = torch.tensor([x_min, x_max])
box_1 = BoxTensor(data_x)

y_min = [1/n for n in range(1, 101)]
y_max = [1 - k for k in reversed(y_min)]
data_y = torch.tensor([y_min, y_max], requires_grad=True)
box_2 = BoxTensor(data_y)

# Intersection of box_1 and box_2
print(hard_intersection(box_1, box_2))

# Gumbel intersection of box_1 and box_2
print(gumbel_intersection(box_1, box_2))

BoxTensor(z=tensor([1.0000, 0.5000, 0.3333, 0.2500, 0.2000, 0.1667, 0.1429, 0.1250, 0.1111,
        0.1000, 0.0909, 0.0833, 0.0769, 0.0714, 0.0667, 0.0625, 0.0588, 0.0556,
        0.0526, 0.0500, 0.0476, 0.0455, 0.0435, 0.0417, 0.0400, 0.0385, 0.0370,
        0.0357, 0.0345, 0.0333, 0.0323, 0.0312, 0.0303, 0.0294, 0.0286, 0.0278,
        0.0270, 0.0263, 0.0256, 0.0250, 0.0244, 0.0238, 0.0233, 0.0227, 0.0222,
        0.0217, 0.0213, 0.0208, 0.0204, 0.0200, 0.0196, 0.0192, 0.0189, 0.0185,
        0.0182, 0.0179, 0.0175, 0.0172, 0.0169, 0.0167, 0.0164, 0.0161, 0.0159,
        0.0156, 0.0154, 0.0152, 0.0149, 0.0147, 0.0145, 0.0143, 0.0141, 0.0139,
        0.0137, 0.0135, 0.0133, 0.0132, 0.0130, 0.0128, 0.0127, 0.0125, 0.0123,
        0.0122, 0.0120, 0.0119, 0.0118, 0.0116, 0.0115, 0.0114, 0.0112, 0.0111,
        0.0110, 0.0109, 0.0108, 0.0106, 0.0105, 0.0104, 0.0103, 0.0102, 0.0101,
        0.0100], grad_fn=<MaximumBackward>),
Z=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

### 4. Box Training
In the following example, we train a simple box `box_2` to require it to be completely contained inside another box
`box_1`. The training loop returns the best `box_2`.

In [79]:
# Training boxes
from box_embeddings.modules.volume.soft_volume import log_soft_volume
from box_embeddings.modules.intersection import hard_intersection

x_min = [-2.0 for n in range(1, 101)]
x_max = [0.0 for k in reversed(x_min)]
data_x = torch.tensor([x_min, x_max], requires_grad=True)
box_1 = BoxTensor(data_x)

y_min = [1/n for n in range(1, 101)]
y_max = [1 - k for k in reversed(y_min)]
data_y = torch.tensor([y_min, y_max], requires_grad=True)
box_2 = BoxTensor(data_y)
learning_rate = 0.1

def train(box_1, box_2, optimizer, epochs=1):
    best_loss = int()
    best_box_2 = None
    for e in range(epochs):
        loss = log_soft_volume(box_2)-log_soft_volume(hard_intersection(box_1, box_2))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if best_loss < loss.item():
            best_loss = loss.item()
            best_box_2 = box_2
        print('Iteration %d, loss = %.4f' % (e, loss.item()))
    return best_box_2

optimizer =  torch.optim.SGD([data_y], lr=learning_rate)
train(box_1, box_2, optimizer, epochs=20)

Iteration 0, loss = 61.7071
Iteration 1, loss = 58.2051
Iteration 2, loss = 54.6527
Iteration 3, loss = 51.0442
Iteration 4, loss = 47.3736
Iteration 5, loss = 43.6348
Iteration 6, loss = 39.8212
Iteration 7, loss = 35.9260
Iteration 8, loss = 31.9714
Iteration 9, loss = 27.9444
Iteration 10, loss = 23.8155
Iteration 11, loss = 19.6192
Iteration 12, loss = 15.3355
Iteration 13, loss = 11.0073
Iteration 14, loss = 6.7032
Iteration 15, loss = 2.5618
Iteration 16, loss = 0.0000
Iteration 17, loss = 0.0000
Iteration 18, loss = 0.0000
Iteration 19, loss = 0.0000


BoxTensor(tensor([[ 9.0841e-01,  3.8041e-01,  2.0479e-01,  1.1735e-01,  6.4819e-02,
          2.9872e-02,  4.9503e-03, -1.3714e-02, -2.8212e-02, -3.9795e-02,
         -4.9260e-02, -5.7138e-02, -6.3795e-02, -6.9492e-02, -7.4423e-02,
         -7.8730e-02, -8.2524e-02, -8.5890e-02, -8.8896e-02, -9.1595e-02,
         -9.4031e-02, -9.6240e-02, -9.8251e-02, -1.0009e-01, -1.0178e-01,
         -1.0333e-01, -1.0476e-01, -1.0608e-01, -1.0730e-01, -1.0844e-01,
         -1.0950e-01, -1.1049e-01, -1.1141e-01, -1.1227e-01, -1.1308e-01,
         -1.1384e-01, -1.1455e-01, -1.1521e-01, -1.1584e-01, -1.1642e-01,
         -1.1698e-01, -1.1750e-01, -1.1798e-01, -1.1844e-01, -1.1887e-01,
         -1.1928e-01, -1.1966e-01, -1.2001e-01, -1.2035e-01, -1.2066e-01,
         -1.2095e-01, -1.2122e-01, -1.2147e-01, -1.2169e-01, -1.2191e-01,
         -1.2210e-01, -1.2227e-01, -1.2242e-01, -1.2256e-01, -1.2268e-01,
         -1.2278e-01, -1.2286e-01, -1.2292e-01, -1.2296e-01, -1.2299e-01,
         -1.2299e-01, -1.229